In [21]:
# First step, import libraries.
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt


In [24]:
# Import the dataset and encode the date
df = pd.read_csv('bitstampUSD_1-min_data_2012-01-01_to_2017-10-20.csv')
df['date'] = pd.to_datetime(df['Timestamp'],unit='s').dt.date
group = df.groupby('date')
Real_Price = group['Weighted_Price'].mean()

In [25]:
# split data
prediction_days = 30
df_train= Real_Price[:len(Real_Price)-prediction_days]
df_test= Real_Price[len(Real_Price)-prediction_days:]

In [26]:
# Data preprocess
training_set = df_train.values
training_set = np.reshape(training_set, (len(training_set), 1))
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
training_set = sc.fit_transform(training_set)
X_train = training_set[0:len(training_set)-1]
y_train = training_set[1:len(training_set)]
X_train = np.reshape(X_train, (len(X_train), 1, 1))

In [32]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import LSTM

In [44]:
# Initialising the RNN
regressor = Sequential()

# Adding the input layer and the LSTM layer
regressor.add(LSTM(units = 4, activation = 'sigmoid', input_shape = (None, 1)))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, batch_size = 5, epochs = 100)


Epoch 1/100
2087/2087 [==============================] - 4s 2ms/step - loss: 0.0226
Epoch 2/100
2087/2087 [==============================] - 3s 1ms/step - loss: 0.0193
Epoch 3/100
2087/2087 [==============================] - 3s 1ms/step - loss: 0.0160
Epoch 4/100
2087/2087 [==============================] - 3s 1ms/step - loss: 0.0127
Epoch 5/100
2087/2087 [==============================] - 3s 1ms/step - loss: 0.0093
Epoch 6/100
2087/2087 [==============================] - 3s 1ms/step - loss: 0.0062
Epoch 7/100
2087/2087 [==============================] - 4s 2ms/step - loss: 0.0037
Epoch 8/100
2087/2087 [==============================] - 4s 2ms/step - loss: 0.0019
Epoch 9/100
2087/2087 [==============================] - 3s 2ms/step - loss: 7.9336e-04
Epoch 10/100
2087/2087 [==============================] - 3s 1ms/step - loss: 2.9722e-04
Epoch 11/100
2087/2087 [==============================] - 3s 1ms/step - loss: 1.2570e-04
Epoch 12/100
2087/2087 [==============================] - 3s 

2087/2087 [==============================] - 3s 1ms/step - loss: 6.4095e-05
Epoch 91/100
2087/2087 [==============================] - 3s 1ms/step - loss: 6.4183e-05
Epoch 92/100
2087/2087 [==============================] - 3s 1ms/step - loss: 6.2069e-05
Epoch 93/100
2087/2087 [==============================] - 3s 1ms/step - loss: 6.2436e-05
Epoch 94/100
2087/2087 [==============================] - 3s 2ms/step - loss: 6.1232e-05
Epoch 95/100
2087/2087 [==============================] - 3s 2ms/step - loss: 6.1577e-05
Epoch 96/100
2087/2087 [==============================] - 3s 2ms/step - loss: 6.1421e-05
Epoch 97/100
2087/2087 [==============================] - 3s 1ms/step - loss: 6.4800e-05
Epoch 98/100
2087/2087 [==============================] - 3s 1ms/step - loss: 6.4013e-05
Epoch 99/100
2087/2087 [==============================] - 3s 1ms/step - loss: 6.0903e-05
Epoch 100/100
2087/2087 [==============================] - 3s 2ms/step - loss: 6.2262e-05


In [45]:
regressor.save("model.h5")

In [46]:
# Making the predictions
test_set = df_test.values
inputs = np.reshape(test_set, (len(test_set), 1))
inputs = sc.transform(inputs)
inputs = np.reshape(inputs, (len(inputs), 1, 1))
predicted_BTC_price = regressor.predict(inputs)
predicted_BTC_price = sc.inverse_transform(predicted_BTC_price)